### Molecular dynamics project notebook template

This is a template to quick-start your new MD project. 

You can use it to document how your systems were setup, simulated, processed, followed by analysis and plots.

In [ ]:
# collect all import in the beginning, don't scatter them through the notebook

# general libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

# Molecular Dynamics analysis libraries
import MDAnalysis as mda
from MDAnalysis import transformations

import lipyphilic as lpp

# custom python libraries
from yamdtools.property_tracker import PropertyAnalyser, RMSDCalculator, LipidPropertyCalculator
from yamdtools.analysis import ZDensity

In [ ]:
# this magic lines are handy if you are loading your custom Pyhton libraries and aditing them as you go
# if reload is not working you might want to restart kernel
%load_ext autoreload
%autoreload 2 

### Auxiliary functions

If have long plotting or analysis functions that you reuse throughout the notebook, you might want to collect them here:

In [ ]:
def plot_results(data):
    """
    Write descriptions, this will help you 
    or anyone else who is using your work
    """

    pass

### Contents

0. [**System assembly**](#section_sim_assembly)

1. [**Stage 1 of analysis**](#section_1)

    1.1 [Analysis of some property](#section_1_1)

    1.2 [Analysis of another property](#section_1_2)

    1.3 [Results](#section_1_3)

3. [**Stage 2 of analysis**](#section_2)

    2.1 [Analysis of some property](#section_1_1)

    2.2 [Analysis of another property](#section_1_2)

    2.3 [Results](#section_2_3)

## System assembly<a id="section_sim_assembly"></a>

Here you can describe how your systems were setup and simulated.

It can also be helpful to iterate through all of the systems to count components in each system and create a table of simulations for your supplementary materials. See example code below:

In [ ]:
# to store data about assembled systems
simulated_systems = []

# let's assume that 'systems' is a lst of paths to simulation directories
system_paths = [...]

for system_path in system_paths:

    # logging progress
    print(f'Analysing {system_path}\n')

    # load your system
    system = mda.Universe(f'{system_path}/mdrun.gro')

    # now let's count components, for example
    # 1. number of lipids in leaflets
    
    #deduce membrane center
    lipids = system.select_atoms('resname POPC')
    midplane = (lipids.positions[:, -1].min() + lipids.positions[:, -1].max()) / 2
    
    # count lipids in leaflets
    l = len(system.select_atoms(f'resname POPC and name PO4 and prop z < {round(midplane, 1)}').residues)
    u = len(system.select_atoms(f'resname POPC and name PO4 and prop z > {round(midplane, 1)}').residues)
    n_lipids = len(system.select_atoms('resname POPC').residues)
    
    # 2. number of water and ions
    water = 'W'
    n_w = len(system.select_atoms(f'resname {water}').residues)

    na, cl = 'NA', 'CL'
    n_Na = len(system.select_atoms(f'name {na}').residues)
    n_Cl = len(system.select_atoms(f'name {cl}').residues)

    # 3. system dimensions
    x, y , z = system.dimensions[:3] / 10

    # 4. total number of particles
    n_atoms = len(system.atoms)

    # add info to list
    # SUGGESTION. you might want to unpack system_path into more 
    # informative tags e.g. protein name, replicate number etc.
    simulated_systems.append([
        system_path,
        n_lipids, l, u,
        n_w, n_Na, n_Cl,
        n_atoms,
        round(x, 1), round(y, 1), round(z, 1) 
    ])

print('\n\n')

# create dataframe 
simulated_systems = pd.DataFrame(
    data = simulated_systems,
    columns = [
        'system_name',
        'n_lipids', 'n_lower_leaflet', 'n_upper_leaflet',
        'Na', 'Cl', 'n_particles',
        'x', 'y', 'z'
    ]
)

In [ ]:
pd.set_option('display.max_rows', None)
simulated_systems

In [ ]:
# save
simulated_systems.to_csv('analysis/table_fo_simulations.csv')

In [ ]:
# reload and combine dataframes
dfs = []
for name, sim in zip(
    ['DMPC64', 'DMPC320 packmol assembly', 'DMPC320 CHARMM-GUI assembly'],
    ['small_membrane', 'bigger_membrane', 'bigger_membrane_charmm_gui']
):
    df = pd.read_csv(f'simulations_benchmarking/data/tutorial/{sim}.csv', index_col=0)
    df['name'] = name
    dfs.append(df)

# concat
df = pd.concat(dfs, ignore_index=True, axis=0)

# save
df.to_csv(f'simulations_benchmarking/data/tutorial/DMPC_data.csv')

# create PropertyAnalyser and fill data
pa = PropertyAnalyser()
pa.data = df

print(df.shape)
df.head()

In [ ]:
# plot and calculate statistics
pa.plot(
    properties_to_plot=['Area per lipid', 'Bilayer thickness'],
    # palette='Set2',
    plot_convergence=True,
    sns_kwargs={'alpha': 0.9}
)
# plt.savefig(f'simulations_benchmarking/plots/DMPC_SIRAH.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
cols = ['Area per lipid', 'Bilayer thickness', 'Order parameter']
pa.data.groupby(['Step_name'], as_index=False)[cols].agg(['mean','std'])

Manual check for the APL calculation:

In [ ]:
for sim, n_lip in zip(['small_membrane', 'bigger_membrane'], [64, 320]):
    pa1 = PropertyAnalyser(
        edr = f'simulations_benchmarking/tutorial/{sim}/production/mdrun.edr'
    )
    pa1.extract_properties()
    apl = ((pa1.data['Box-X'].iloc[5000:].mean() * 10) **2) / n_lip
    print(f"in {sim} simulation APL is {round(apl, 2)}")

In [ ]:
rep1/production

Bigger box provides more stable dimensions. 

Looks like box size matters a lot. Bigger box yields more realistic values.
An additional question for more simulations could be: **How does this compare to other force-fields?**

Although in the big box the bilayer is more curved, It could affect membrane thickness calculation which could be dependent on binning. **Let's explore that:**

In [ ]:
system = mda.Universe(
    'simulations_benchmarking/tutorial/bigger_membrane/production/mdrun.tpr',
    'simulations_benchmarking/tutorial/bigger_membrane/production/mdrun.xtc'
)

ag = system.atoms
workflow = [
    transformations.unwrap(ag),
    transformations.center_in_box(system.select_atoms('resname CMM')),
    transformations.wrap(ag, compound='residues')
    ]	
system.trajectory.add_transformations(*workflow)

In [ ]:
# fixed leaflet assignment binning
# increasing membrane thickness binning

# asssign leaflets 
n_bins_leaflets  = int(system.dimensions[0] // 10)

leaflets = lpp.leaflets.assign_leaflets.AssignLeaflets(
  universe = system,
  lipid_sel = 'name BFO',
  n_bins = n_bins_leaflets 
)

leaflets.run(
    step = 250,
    verbose = True
)

# Membrane thickness
mt_dat = []
for bin_len in [5, 10, 15, 20, 25]:
    
    n_bins_thickness = int(system.dimensions[0] // bin_len)

    # compute thickness
    memb_thickness = lpp.analysis.MembThickness(
        universe = system,
        leaflets = leaflets.leaflets,
        lipid_sel = 'name BFO',
        n_bins = n_bins_thickness
    )
    
    memb_thickness.run(
        step = 250,
        verbose = True
    )

    mt_dat.append(memb_thickness.memb_thickness)

In [ ]:
# plot
time = np.arange(0, 1000 + 0.1, 0.1 * 250)
for dat, bin_len in zip(mt_dat, [5, 10, 15, 20, 25]):
    plt.plot(time, dat, label=bin_len)
plt.legend()
plt.show()

In [ ]:
# fixed membrane thickness binning
# increasing leaflet assignment binning

# Membrane thickness
mt_dat = []
for bin_len in [3, 5, 10, 15, 20, 25]:

    # asssign leaflets 
    n_bins_leaflets  = int(system.dimensions[0] // bin_len)
    
    leaflets = lpp.leaflets.assign_leaflets.AssignLeaflets(
      universe = system,
      lipid_sel = 'name BFO',
      n_bins = n_bins_leaflets 
    )
    
    leaflets.run(
        step = 250,
        verbose = True
    )

    n_bins_thickness = int(system.dimensions[0] // 20)

    # compute thickness
    memb_thickness = lpp.analysis.MembThickness(
        universe = system,
        leaflets = leaflets.leaflets,
        lipid_sel = 'name BFO',
        n_bins = n_bins_thickness
    )
    
    memb_thickness.run(
        step = 250,
        verbose = True
    )

    mt_dat.append(memb_thickness.memb_thickness)

In [ ]:
# plot
time = np.arange(0, 1000 + 0.1, 0.1 * 250)
for dat, bin_len in zip(mt_dat, [3, 5, 10, 15, 20, 25]):
    plt.plot(time, dat, label=bin_len)
plt.legend()
plt.show()

Looks like the parameters I'm using for binning are actually sensinble.

## 1. Stage 1 of analysis<a id="section_1"></a>

### 1.1 Analysis of some property<a id="section_1_1"></a>

In [ ]:
############
# YOUR CODE
############

### 1.2 Analysis of another property<a id="section_1_2"></a>

In [ ]:
############
# YOUR CODE
############

### 1.3 Results<a id="section_1_3"></a>

Describe you conclusions and observations in brief or in the manner of the results section of a paper.

## 2. Stage 2 of analysis<a id="section_2"></a>

### 2.1 Analysis of some property<a id="section_2_1"></a>

In [ ]:
############
# YOUR CODE
############

### 2.2 Analysis of another property<a id="section_2_2"></a>

In [ ]:
############
# YOUR CODE
############

### 1.3 Results<a id="section_2_3"></a>

...

In [ ]:
# read RMSF data
rmsf_data = []
for rep in range(1, 4):
    rmsf_data.append(pd.read_csv(
        f"simulations_benchmarking/popc_bilayer_simulations/OmpX/charmm36m/rep{rep}/production/rmsf_CA.xvg",
        sep='\s+',
        header=None,
        names=['step','metric']
    ))

# plot
fig = RMSFPlot(
    rmsf_data,
    labels = [f"rep{rep}" for rep in range(1, 4)],
    title=f"OmpX Ca RMSF in POPC membrane",
    x_lab="Residue index",
    y_lab="RMSF, nm",
    n_chains=1
)
plt.show()

In [ ]:
# pring residue id spans with RMSF above 0.2 nm
mobile_residues = []
for i, rmsf in enumerate(rmsf_data[0].metric.values):
    if rmsf >= 0.2:
        mobile_residues.append(i + 1)
print(mobile_residues)

**resid 14-18 50-57 92-101**

We will calculate RMSD for backbone and Ca atoms including and excluding these spans.

In [ ]:
plot_by_ff_and_rep(pa.data, prop='RMSD_Ca')
plt.show()

In [ ]:
plot_by_ff_and_rep(pa.data, prop='RMSD_Ca_core')
plt.savefig(f'simulations_benchmarking/plots/OmpX_popc_bilayer_RMSD.jpg', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# combine data
dfs = []
for ff in ['charmm36m', 'SIRAH', 'M2', 'M3']:
    for rep in range(1, 4):
        df = pd.read_csv(
            f'simulations_benchmarking/mixed_bilayer_simulations/OmpX/{ff}/rep{rep}/rmsd_data.csv',
            index_col=0
        )
        df['Step_name'] = ff
        df['rep'] = rep
        dfs.append(df)

# concat
df = pd.concat(dfs, ignore_index=True, axis=0)

# save
df.to_csv(f'simulations_benchmarking/data/OmpX_mixed_rmsd_data.csv')

# create PropertyAnalyser and fill data
pa = PropertyAnalyser()
pa.data = df

print(df.shape)
df.head()

In [ ]:
plot_by_ff_and_rep(pa.data, prop='RMSD_Ca')
plt.show()

In [ ]:
plot_by_ff_and_rep(pa.data, prop='RMSD_Ca_core')
plt.savefig(f'simulations_benchmarking/plots/OmpX_mixed_bilayer_RMSD.jpg', dpi=300, bbox_inches='tight')
plt.show()